<h1>Курсовая работа</h1>
<h2>за 5 семестр</h2>
<h2>по теме: "Разработка CRUD-приложения на Python с использованием 
    SQLAlchemy ORM и SQLite"</h2>
<h2>Предметная область: "Страховая компания"</h2>
<h3>Выполнил: студент 3-го курса гр. ПИН-221 Федоров Вячеслав</h3>

<h4>1 Подключение к SQLite и подготовка окружения</h4>

Подключение необходимых библиотек

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date, Float
from sqlalchemy import func, case, desc, and_, extract
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from datetime import datetime, timedelta
from faker import Faker
import random
from sqlalchemy.exc import SQLAlchemyError
from tabulate import tabulate

Подключение к SQLite с автоматическим созданием файла БД

In [2]:
databaseName = 'insurance_company'

# основные настройки БД
engine = create_engine(f'sqlite:///{databaseName}.db')
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

# проверка подключения
try:
    with engine.connect() as conn:
        print("Подлкючение к SQLite произведено успешно!")
except Exception as e:
    print(f"Ошибка подключения: {e}")

Подлкючение к SQLite произведено успешно!


<h4>2 Создание схемы базы данных</h4>

Создание класса "Filial"

In [3]:
class Filial(Base):
    __tablename__ = 'filial' 

    idFilial = Column(Integer, primary_key=True)
    title = Column(String)  
    address = Column(String)
    phoneNumber = Column(String)

    # Связь с агентами и договорами (один ко многим)
    agents = relationship("Agent", back_populates="filial")
    belays = relationship("Belay", back_populates="filial")

    def __init__(self, title: str, address: str, phoneNumber: str):
        self.title = title
        self.address = address
        self.phoneNumber = phoneNumber

    def __info__(self):
        return f"Страховая компания: [ID: {self.idFilial} Наименование: {self.title}, " \
               f"Адрес: {self.address}, Номер телефона: {self.phoneNumber}]"
    
    def __str__(self):
        return self.__info__()
    

Создание класса "Agent"

In [4]:
class Agent(Base):
    __tablename__ = 'agent' 

    # Инициализируем поля
    idAgent = Column(Integer, primary_key=True)
    fullName = Column(String)  
    address = Column(String)
    phoneNumber = Column(String)
    salary = Column(Integer)
    idFilial = Column(Integer, ForeignKey('filial.idFilial'))
    
    # Связь с филиалом (многие к одному)
    filial = relationship("Filial", back_populates="agents")
    # Связь с договором (один ко многим)
    belays = relationship("Belay", back_populates="agent")

    # Добавляем дополнительные поля для компонентов имени
    name = Column(String)
    surname = Column(String)
    lastname = Column(String)

    # Создаем конструктор объекта класс
    def __init__(self, fullName: list[str], address: str, phoneNumber: str, salary: int, idFilial: int):
        self.surname = fullName[0]
        self.name = fullName[1]
        self.lastname = fullName[2]
        self.fullName = ' '.join(fullName)
        self.address = address
        self.phoneNumber = phoneNumber
        self.salary = salary
        self.idFilial = idFilial

    # отладочный метод для строкового вывода экзмепляра класса
    def __repr__(self):
        return f"Страховой агент [ID: {self.idAgent}, ФИО: {self.surname} {self.name} {self.lastname}, " \
               f"Адрес: {self.address}, Номер телефона: {self.phoneNumber}, " \
               f"Зарплата (руб): {self.salary}, ID филиала: {self.idFilial}]"
    
    # метод-обёртка для удобного вывода
    def __str__(self):
        return self.__repr__()

Cоздание класса Client

In [5]:
class Client(Base):
    __tablename__ = 'client' 

    idClient = Column(Integer, primary_key=True)
    fullName = Column(String)  
    age = Column(Integer)
    address = Column(String)
    phoneNumber = Column(String)

    name = Column(String)
    surname = Column(String)
    lastname = Column(String)

    # Связь с договорами (один ко многим)
    belays = relationship("Belay", back_populates="client")

    def __init__(self, fullName: list[str], age: int, address: str, phoneNumber: str):
        self.surname = fullName[0]
        self.name = fullName[1]
        self.lastname = fullName[2]
        self.fullName = ' '.join(fullName)
        self.age = age
        self.address = address
        self.phoneNumber = phoneNumber

    def __repr__(self):
        return f"Клиент: [ID: {self.idClient}, ФИО: {self.surname} {self.name} {self.lastname}, Возраст: {self.age}, " \
               f"Адрес: {self.address}, Номер телефона: {self.phoneNumber}]"
    
    def __str__(self):
        return self.__repr__()

Cоздание класса "Belay"

In [6]:
class Belay(Base):
    __tablename__ = 'belay'

    idBelay = Column(Integer, primary_key=True)
    idClient = Column(Integer, ForeignKey('client.idClient'))
    idAgent = Column(Integer, ForeignKey('agent.idAgent'))
    idFilial = Column(Integer, ForeignKey('filial.idFilial'))
    date = Column(Date)  
    insureType = Column(String)
    tariffRate = Column(Float)
    totalAmount = Column(Float)
    monthlyPayment = Column(Float)

    # Связи (многие к одному)
    client = relationship("Client", back_populates="belays")  # Клиент имеет много договоров
    agent = relationship("Agent", back_populates="belays")    # Агент имеет много договоров
    filial = relationship("Filial", back_populates="belays") # Филиал имеет много договоров

    def __init__(self, idClient: int, idAgent: int, idFilial: int, date: str, 
                 insureType: str, tariffRate: float, totalAmount: float, monthlyPayment: float):
        self.idClient = idClient
        self.idAgent = idAgent
        self.idFilial = idFilial
        self.date = date
        self.insureType = insureType
        self.tariffRate = tariffRate
        self.totalAmount = totalAmount
        self.monthlyPayment = monthlyPayment

    def __repr__(self):
        return f"Страховое свидетельство [ID: {self.idBelay}, ID Клиента: {self.idClient}, ID Агента: {self.idAgent}, ID Филиала: {self.idFilial}, " \
               f"Дата заключения: {self.date}, Тип страхования: {self.insureType}, Процентная ставка: {self.tariffRate}, " \
               f"Сумма выплаты: {self.totalAmount}, Ежемесячный платеж: {self.monthlyPayment}]"
    
    def __str__(self):
        return self.__repr__()

Создание таблиц в БД

In [7]:
Base.metadata.create_all(engine)

<h4>3 Заполнение БД тестовыми данными</h4>

Добавление тестовых данных в БД с помощью методов-конструкторов

In [8]:
try:
    # Создание и добавление в БД первого филиала
    first_filial = Filial(
        title='Страховая контора',
        address='ул. Беговая, д. 11',
        phoneNumber='+73812992233'
    )
    session.add(first_filial)
    print(first_filial)

    # Создание и добавление в БД первого агента
    first_agent = Agent(
        fullName=['Иванов', 'Иван', 'Иванович'],
        address='ул. Кропоткинская, д. 12, кв. 45',
        phoneNumber='+79991112233',
        salary=125000,
        idFilial=1
    )
    session.add(first_agent)
    print(first_agent)

    # Создание и добавление в БД первого клиента
    first_client = Client(
        fullName=['Веселовская', 'Мария', 'Павловна'],
        age=25,
        address='ул. Моховая, д. 234, кв. 112',
        phoneNumber='+70004445599'
    )
    session.add(first_client)
    print(first_client)

    # Создание и добавление в БД первого страхового договора
    first_belay = Belay(
        idClient=1,
        idAgent=1,
        idFilial=1,
        date=datetime.strptime('12-09-2025', '%d-%m-%Y').date(),  # Преобразуем строку в date
        insureType='Автострахование',
        tariffRate=0.33,
        totalAmount=32345.15,
        monthlyPayment=570.11
    )
    session.add(first_belay)
    print(first_belay)

    # Сохранение изменений в БД
    session.commit()
    print("\nТестовые данные успешно созданы и добавлены в БД!")

except Exception as e:
    session.rollback()
    print(f"\nОшибка при создании тестовых данных: {e}")

Страховая компания: [ID: None Наименование: Страховая контора, Адрес: ул. Беговая, д. 11, Номер телефона: +73812992233]
Страховой агент [ID: None, ФИО: Иванов Иван Иванович, Адрес: ул. Кропоткинская, д. 12, кв. 45, Номер телефона: +79991112233, Зарплата (руб): 125000, ID филиала: 1]
Клиент: [ID: None, ФИО: Веселовская Мария Павловна, Возраст: 25, Адрес: ул. Моховая, д. 234, кв. 112, Номер телефона: +70004445599]
Страховое свидетельство [ID: None, ID Клиента: 1, ID Агента: 1, ID Филиала: 1, Дата заключения: 2025-09-12, Тип страхования: Автострахование, Процентная ставка: 0.33, Сумма выплаты: 32345.15, Ежемесячный платеж: 570.11]

Тестовые данные успешно созданы и добавлены в БД!


Автоматическое заполнение БД тестовыми данными с помощью Faker

Заполнение таблицы filial

In [9]:
try:
    titles = ['Страховая контора', 'ИНГОССТРАХ', 'РЕСО-Гарантия', 'Сбер-страхование', 'ВСК', 'СОГАЗ']
    filialIds = []

    faker = Faker('ru-RU')

    # словарь счётчиков(сколько раз было добавлено конкретное наименование)
    titles_temp = {
        'Страховая контора': 0, 
        'ИНГОССТРАХ': 0, 
        'РЕСО-Гарантия': 0, 
        'Сбер-страхование': 0, 
        'ВСК': 0, 
        'СОГАЗ': 0
    }

    for i in range(1, 31):
        # Выбираем случайное название
        chosen_title = faker.random.choice(titles)
        
        # Увеличиваем счетчик для выбранного названия
        titles_temp[chosen_title] += 1
        
        # Если название повторяется больше 1 раза, добавляем номер
        if titles_temp[chosen_title] > 1:
            final_title = f'{chosen_title} ({titles_temp[chosen_title]})'
        else:
            final_title = chosen_title

        filial = Filial(
            title=final_title,
            address=faker.address(),
            phoneNumber=faker.phone_number()
        )

        session.add(filial)
        session.flush()
        filialIds.append(filial.idFilial)

    session.commit()
    print(f'Успешно добавлено {len(filialIds)} филиалов!')
    print("Статистика по названиям:")
    for title, count in titles_temp.items():
        print(f"  {title}: {count} филиалов")

except Exception as e:
    session.rollback()
    print(f"Ошибка при добавлении филиалов: {str(e)}")

Успешно добавлено 30 филиалов!
Статистика по названиям:
  Страховая контора: 4 филиалов
  ИНГОССТРАХ: 5 филиалов
  РЕСО-Гарантия: 3 филиалов
  Сбер-страхование: 6 филиалов
  ВСК: 8 филиалов
  СОГАЗ: 4 филиалов


Заполнение таблицы agent

In [10]:
try:
    agentIds = []
    agentFilialMappingDict = {}  # словарь для хранения связи агент-филиал
    
    faker = Faker('ru-RU')

    for i in range(1, 51):
        id_filial = random.choice(filialIds)
        rec = Agent(
            fullName=faker.name().split(' '),  
            address=faker.address(),
            phoneNumber=faker.phone_number(),
            salary=random.randint(50000, 150000), 
            idFilial=id_filial
        )
        session.add(rec)
        session.flush()
        agentIds.append(rec.idAgent)
        agentFilialMappingDict[rec.idAgent] = id_filial  # Сохраняем связь

    session.commit()
    print(f'Успешно добавлено {len(agentIds)} агентов!')

except Exception as e:
    session.rollback()
    print(f"Ошибка при добавлении агентов: {str(e)}")

Успешно добавлено 50 агентов!


Заполнение таблицы client

In [11]:
try:
    clientIds = []
    
    faker = Faker('ru-RU')

    for i in range(1, 101):
        client = Client(
            fullName=faker.name().split(' '),
            age=random.randint(18, 80),
            address=faker.address(),
            phoneNumber=faker.phone_number(),
        )
        session.add(client)
        session.flush()
        clientIds.append(client.idClient)

    session.commit()
    print(f'Успешно добавлено {len(clientIds)} клиентов!')

except Exception as e:
    session.rollback()
    print(f"Ошибка при добавлении клиентов: {str(e)}")

Успешно добавлено 100 клиентов!


Заполнение таблицы belay

In [12]:
try:
    insureTypes = [
        'Автострахование', 'Страхование жизни', 'Спортивная страховка', 'Страхование недвижимости',
        'Обязательное медицинское страхование', 'Страхование для военнослужащих', 'Страхование гражданской ответственности',
        'Бизнес-страхование', 'Туристическое страхование'
    ]
    
    faker = Faker('ru-RU')

    for i in range(1, 151):
        # добавление полей id с учётом связей между таблицами
        id_client = random.choice(clientIds)
        id_agent = random.choice(agentIds)
        id_filial = agentFilialMappingDict[id_agent]
        
        # генерация параметров страхового договора
        tariff_rate = round(random.uniform(0.5, 5.0), 2)  # Тарифная ставка 0.5% - 5%
        total_amount = round(random.uniform(100000.00, 1000000.00), 2)  # Общая сумма 100к - 1м
        monthly_payment = round(total_amount * tariff_rate / 100 / 12, 2)  # Ежемесячный платеж
        
        # генерация даты заключения договора
        start_date = datetime.now() - timedelta(days=730)
        random_date = faker.date_between(start_date=start_date, end_date='today')
        
        belay = Belay(
            idClient=id_client,
            idAgent=id_agent,
            idFilial=id_filial,
            date=random_date,
            insureType=random.choice(insureTypes),
            tariffRate=tariff_rate,
            totalAmount=total_amount,
            monthlyPayment=monthly_payment
        )
        session.add(belay)

    session.commit()
    print(f'Успешно добавлено {i} договоров!')

except Exception as e:
    session.rollback()
    print(f"Ошибка при добавлении договоров: {str(e)}")

Успешно добавлено 150 договоров!


<h4>4 Чтение данных из БД</h4>

Чтение данных из таблицы filial

In [13]:
try:
    count = 0

    # получение всех записей из таблицы filial
    filials = session.query(Filial).all()

    # вывод на экран 10 записей из таблицы
    print("Данные из таблицы filial:")
    for filial in filials:
        count += 1
        if count <= 10:
            print(filial)

except SQLAlchemyError as e:
    # обработка исключений SQLAlchemy
    print("Произошла ошибка при работе с базой данных:")
    print(str(e))

except Exception as e:
    # обработка любых других исключений
    print("Произошла ошибка:")
    print(str(e))

finally:
    # закрытие сессии
    session.close()

Данные из таблицы filial:
Страховая компания: [ID: 1 Наименование: Страховая контора, Адрес: ул. Беговая, д. 11, Номер телефона: +73812992233]
Страховая компания: [ID: 2 Наименование: ВСК, Адрес: п. Усть-Кут, пр. Минский, д. 770 стр. 1, 786939, Номер телефона: +7 (531) 023-43-61]
Страховая компания: [ID: 3 Наименование: РЕСО-Гарантия, Адрес: клх Саратов, наб. Гоголя, д. 8/1, 230950, Номер телефона: +7 520 569 4984]
Страховая компания: [ID: 4 Наименование: СОГАЗ, Адрес: с. Курганинск, ш. Осенное, д. 6/7, 012175, Номер телефона: 8 810 773 1720]
Страховая компания: [ID: 5 Наименование: ВСК (2), Адрес: ст. Дмитров, наб. Металлургов, д. 51, 300093, Номер телефона: +70729379335]
Страховая компания: [ID: 6 Наименование: Страховая контора, Адрес: ст. Партизанск, бул. Макаренко, д. 8/2, 437490, Номер телефона: 8 521 143 22 60]
Страховая компания: [ID: 7 Наименование: Сбер-страхование, Адрес: клх Далматово, бул. К.Маркса, д. 831, 876452, Номер телефона: 8 783 136 9887]
Страховая компания: [ID: 8

Чтение данных из таблицы agent

In [14]:
try:
    count = 0

    # получение всех записей из таблицы filial
    agents = session.query(Agent).all()

    # вывод на экран 10 записей из таблицы
    print("Данные из таблицы agent:")
    for rec in agents:
        count += 1
        if count <= 10:
            print(rec)

except SQLAlchemyError as e:
    # обработка исключений SQLAlchemy
    print("Произошла ошибка при работе с базой данных:")
    print(str(e))

except Exception as e:
    # обработка любых других исключений
    print("Произошла ошибка:")
    print(str(e))

finally:
    # закрытие сессии
    session.close()

Данные из таблицы agent:
Страховой агент [ID: 1, ФИО: Иванов Иван Иванович, Адрес: ул. Кропоткинская, д. 12, кв. 45, Номер телефона: +79991112233, Зарплата (руб): 125000, ID филиала: 1]
Страховой агент [ID: 2, ФИО: Кира Филипповна Калинина, Адрес: клх Сунтар, ш. 40 лет Октября, д. 76 к. 4/3, 422455, Номер телефона: 8 (035) 200-8692, Зарплата (руб): 81772, ID филиала: 4]
Страховой агент [ID: 3, ФИО: Пахомова Марина Никифоровна, Адрес: г. Тольятти, алл. Мусы Джалиля, д. 6/1, 738753, Номер телефона: +7 (660) 273-2817, Зарплата (руб): 146848, ID филиала: 2]
Страховой агент [ID: 4, ФИО: Герасимова Жанна Макаровна, Адрес: с. Ноглики, бул. Красных Партизан, д. 1/5 стр. 5/5, 049814, Номер телефона: +7 (252) 650-6678, Зарплата (руб): 99989, ID филиала: 13]
Страховой агент [ID: 5, ФИО: Пелагея Вениаминовна Игнатова, Адрес: п. Ивдель, наб. Коммуны, д. 398, 011890, Номер телефона: +7 233 858 15 68, Зарплата (руб): 111881, ID филиала: 29]
Страховой агент [ID: 6, ФИО: Иларион Давыдович Борисов, Адре

Чтение данных из таблицы client

In [15]:
try:
    count = 0

    # получение всех записей из таблицы filial
    clients = session.query(Client).all()

    # вывод на экран 15 записей из таблицы
    print("Данные из таблицы agent:")
    for client in clients:
        count += 1
        if count <= 15:
            print(client)

except SQLAlchemyError as e:
    # обработка исключений SQLAlchemy
    print("Произошла ошибка при работе с базой данных:")
    print(str(e))

except Exception as e:
    # обработка любых других исключений
    print("Произошла ошибка:")
    print(str(e))

finally:
    # закрытие сессии
    session.close()

Данные из таблицы agent:
Клиент: [ID: 1, ФИО: Веселовская Мария Павловна, Возраст: 25, Адрес: ул. Моховая, д. 234, кв. 112, Номер телефона: +70004445599]
Клиент: [ID: 2, ФИО: Парамон Гавриилович Исаков, Возраст: 75, Адрес: к. Энгельс, пр. Кольцова, д. 98 стр. 37, 643735, Номер телефона: 87010139883]
Клиент: [ID: 3, ФИО: Дементий Иларионович Борисов, Возраст: 29, Адрес: с. Лянтор, пр. Родниковый, д. 852 стр. 81, 640794, Номер телефона: +7 (805) 867-51-53]
Клиент: [ID: 4, ФИО: Орест Жанович Волков, Возраст: 64, Адрес: ст. Амдерма, ул. Подгорная, д. 126, 548653, Номер телефона: +7 (290) 954-4040]
Клиент: [ID: 5, ФИО: Твердислав Демьянович Терентьев, Возраст: 74, Адрес: г. Терскол, ш. Энергетиков, д. 753, 224238, Номер телефона: +7 234 665 4475]
Клиент: [ID: 6, ФИО: Анжелика Романовна Дьячкова, Возраст: 57, Адрес: ст. Мураши, бул. Нахимова, д. 3 стр. 128, 320361, Номер телефона: 8 (602) 856-10-11]
Клиент: [ID: 7, ФИО: Орлова Оксана Наумовна, Возраст: 25, Адрес: г. Шерегеш, алл. Вахитова, д

Чтение данных из таблицы belay

In [16]:
try:
    count = 0

    # получение всех записей из таблицы filial
    belays = session.query(Belay).all()

    # вывод на экран 20 записей из таблицы
    print("Данные из таблицы agent:")
    for belay in belays:
        count += 1
        if count <= 20:
            print(belay)

except SQLAlchemyError as e:
    # обработка исключений SQLAlchemy
    print("Произошла ошибка при работе с базой данных:")
    print(str(e))

except Exception as e:
    # обработка любых других исключений
    print("Произошла ошибка:")
    print(str(e))

finally:
    # закрытие сессии
    session.close()

Данные из таблицы agent:
Страховое свидетельство [ID: 1, ID Клиента: 1, ID Агента: 1, ID Филиала: 1, Дата заключения: 2025-09-12, Тип страхования: Автострахование, Процентная ставка: 0.33, Сумма выплаты: 32345.15, Ежемесячный платеж: 570.11]
Страховое свидетельство [ID: 2, ID Клиента: 9, ID Агента: 18, ID Филиала: 19, Дата заключения: 2024-02-24, Тип страхования: Страхование гражданской ответственности, Процентная ставка: 0.75, Сумма выплаты: 781724.12, Ежемесячный платеж: 488.58]
Страховое свидетельство [ID: 3, ID Клиента: 54, ID Агента: 50, ID Филиала: 24, Дата заключения: 2024-04-15, Тип страхования: Автострахование, Процентная ставка: 1.56, Сумма выплаты: 198287.7, Ежемесячный платеж: 257.77]
Страховое свидетельство [ID: 4, ID Клиента: 23, ID Агента: 35, ID Филиала: 12, Дата заключения: 2024-02-23, Тип страхования: Страхование недвижимости, Процентная ставка: 1.42, Сумма выплаты: 433983.62, Ежемесячный платеж: 513.55]
Страховое свидетельство [ID: 5, ID Клиента: 11, ID Агента: 26, I

<h4>5 Обновление и изменение данных в БД</h4>

Обновление поля age в таблице client

In [17]:
tableName = 'client'

try:
    # вывод записи до обновления
    clientToUpdate = session.query(Client).filter(Client.idClient == 1).first()
    print(clientToUpdate)

    # обновление поля "возраст"
    if clientToUpdate:
        clientToUpdate.age = random.randint(18, 80)
        session.commit()

except SQLAlchemyError as e:
    session.rollback()
    print(f"Ошибка при обновлении данных в таблице {tableName}: {e}")

try:
    # вывод записи после обновления
    clientUpdated = session.query(Client).filter(Client.idClient == 1).first()
    print(clientUpdated)
    session.close()

except SQLAlchemyError as e:
    session.rollback()
    print(f"Ошибка при работе с таблицей {tableName}: {e}")

Клиент: [ID: 1, ФИО: Веселовская Мария Павловна, Возраст: 25, Адрес: ул. Моховая, д. 234, кв. 112, Номер телефона: +70004445599]
Клиент: [ID: 1, ФИО: Веселовская Мария Павловна, Возраст: 31, Адрес: ул. Моховая, д. 234, кв. 112, Номер телефона: +70004445599]


Создание функции для вывода последних N-записей в виде таблицы

In [18]:
def printLastRecords(modelClass, num):
    try:
        # нахождение первичного ключа
        primary_key = None
        for column in modelClass.__table__.columns:
            if column.primary_key:
                primary_key = column.name
                break
        
        if not primary_key:
            print("Не найден первичный ключ")
            return
        
        # получение последних записей
        lastRecs = session.query(modelClass).order_by(getattr(modelClass, primary_key).desc()).limit(num).all()
        
        # получение названий колонок в таблице
        recData = []
        for rec in lastRecs:
            row = []
            for column in modelClass.__table__.columns:
                row.append(getattr(rec, column.name))
            recData.append(row)

        recData.reverse()
        
        headers = [column.name for column in modelClass.__table__.columns]
        
        print(f"\nПоследние {num} записей из {modelClass.__tablename__}:")
        print(tabulate(recData, headers=headers, tablefmt="grid"))
        
    except Exception as e:
        print(f"Ошибка: {str(e)}")

Добавление записи в таблицу agent и вывод последних 5 записей

In [19]:
tableName = 'agent'

try:
    # добавление новой записи
    newAgent = Agent(
        fullName=['Федоров', 'Федор', 'Федорович'],
        address='ул. Кропоткинская, д. 12, кв. 45',
        phoneNumber='+79991112233',
        salary=125000,
        idFilial=11
    )
    session.add(newAgent)
    print(newAgent)

    # выводим последние 5 записей в таблице
    printLastRecords(Agent, 5)

except SQLAlchemyError as e:
    session.rollback()
    print(f"Ошибка при добавлении новой записи в таблицу {tableName}: {e}")

Страховой агент [ID: None, ФИО: Федоров Федор Федорович, Адрес: ул. Кропоткинская, д. 12, кв. 45, Номер телефона: +79991112233, Зарплата (руб): 125000, ID филиала: 11]

Последние 5 записей из agent:
+-----------+-------------------------------+------------------------------------------------------------------+--------------------+----------+------------+--------------+-----------+------------+
|   idAgent | fullName                      | address                                                          | phoneNumber        |   salary |   idFilial | name         | surname   | lastname   |
+===========+===============================+==================================================================+====================+==========+============+==============+===========+============+
|        48 | Елизавета Владимировна Бурова | п. Зеленогорск (Ленин.), алл. Слободская, д. 78, 262487          | +78036502108       |   135011 |         14 | Владимировна | Елизавета | Бурова     |
+--------

Удаление записи в таблице agent

In [20]:
try:
    # получаем последнюю запись 
    agentToDelete = session.query(Agent).order_by(Agent.idAgent.desc()).first()
    
    if agentToDelete:
        session.delete(agentToDelete)
        session.commit()
        print(f"Последняя запись из таблицы Agent успешно удалена (ID: {agentToDelete.idAgent}).")
    else:
        print("В таблице Agent нет записей для удаления.")

    printLastRecords(Agent, 5)
    session.close()

except SQLAlchemyError as e:
    session.rollback()
    print(f"Ошибка при удалении записи из таблицы Agent: {e}")

Последняя запись из таблицы Agent успешно удалена (ID: 52).

Последние 5 записей из agent:
+-----------+-------------------------------+------------------------------------------------------------------+--------------------+----------+------------+--------------+-----------+------------+
|   idAgent | fullName                      | address                                                          | phoneNumber        |   salary |   idFilial | name         | surname   | lastname   |
+===========+===============================+==================================================================+====================+==========+============+==============+===========+============+
|        47 | Фокина Майя Николаевна        | д. Вологда, пр. Лермонтова, д. 967 стр. 820, 210641              | +7 296 246 2196    |    90124 |          5 | Майя         | Фокина    | Николаевна |
+-----------+-------------------------------+------------------------------------------------------------------+-----

<h4>6 Создание и выполнение сложных запросов на выборку</h4>

Группировка договоров по агентам по определеному типу страхования в порядке убывания общей суммы договора 

In [21]:
# Функция для получения договоров по определенному типу страхования
def getBelaysByInsuranceType(session, insurance_type):
    result = (
        session.query(
            Belay,
            Agent.fullName,  
            Agent.phoneNumber,  
            Filial.title  
        )
        .join(Agent, Agent.idAgent == Belay.idAgent)  
        .join(Filial, Filial.idFilial == Belay.idFilial)  
        .filter(Belay.insureType == insurance_type)
        .order_by(Belay.totalAmount.desc())
        .all()
    )
    return result

try:
    result = getBelaysByInsuranceType(session, "Автострахование")

    # Подготавливаем данные для таблицы
    table_data = []
    for belay, agent_name, phone, filial_name in result:
        table_data.append([
            filial_name,        # Наименование фирмы (филиала)
            agent_name,         # ФИО агента
            phone,              # Номер телефона
            belay.totalAmount,  # Сумма договора
            belay.idFilial,     # ID филиала
            belay.insureType    # Тип страхования
        ])
    
    headers = ["Наименование филиала", "ФИО агента", "Номер телефона", "Сумма договора", "ID филиала", "Тип страхования"]
        
    print(f"\nДоговоры по типу 'Автострахование':")
    print(tabulate(table_data[:10], headers=headers, tablefmt="grid", floatfmt=".2f"))

except SQLAlchemyError as e:
    print("Произошла ошибка при выполнении запроса:")
    print(str(e))

except Exception as e:
    print("Произошла ошибка:")
    print(str(e))

finally:
    session.close()


Договоры по типу 'Автострахование':
+------------------------+------------------------------+--------------------+------------------+--------------+-------------------+
| Наименование филиала   | ФИО агента                   | Номер телефона     |   Сумма договора |   ID филиала | Тип страхования   |
+========================+==============================+====================+==================+==============+===================+
| Сбер-страхование (3)   | Зинаида Ефимовна Ширяева     | 8 363 443 1415     |        887960.74 |           18 | Автострахование   |
+------------------------+------------------------------+--------------------+------------------+--------------+-------------------+
| ВСК (7)                | Елена Федоровна Орехова      | +7 302 560 2803    |        823111.40 |           22 | Автострахование   |
+------------------------+------------------------------+--------------------+------------------+--------------+-------------------+
| ИНГОССТРАХ             | Казак

Получение количества договоров и общей суммы договоров по каждому агенту

In [22]:
def getAgentsStats(session):
    result = (
        session.query(
            Agent.idAgent,
            Agent.fullName,
            func.count(Belay.idBelay).label('contracts_count'),  # Количество договоров
            func.sum(Belay.totalAmount).label('total_sales')     # Общая сумма
        )
        .join(Belay, Belay.idAgent == Agent.idAgent)
        .group_by(Agent.idAgent, Agent.fullName)  # Группировка по агенту
        .order_by(desc('total_sales'))  # Сортировка по сумме продаж
        .all()
    )
    return result

try:
    result = getAgentsStats(session)

    table_data = []
    for agent_id, agent_name, contracts_count, total_sales in result: 
        table_data.append([
            agent_id,
            agent_name,         # ФИО агента
            contracts_count,    # Количество договоров
            total_sales         # Общая сумма продаж
        ])
    
    headers = ["ФИО агента", "Количество договоров", "Общая сумма продаж"]
        
    print(f"\nСтатистика по агентам:")
    print(tabulate(table_data[:10], headers=headers, tablefmt="grid", floatfmt=".2f"))

except SQLAlchemyError as e:
    print("Произошла ошибка при выполнении запроса:")
    print(str(e))

except Exception as e:
    print("Произошла ошибка:")
    print(str(e))

finally:
    session.close()


Статистика по агентам:
+----+-------------------------------+------------------------+----------------------+
|    | ФИО агента                    |   Количество договоров |   Общая сумма продаж |
+====+===============================+========================+======================+
| 12 | Ульяна Владиславовна Быкова   |                      7 |           4511937.70 |
+----+-------------------------------+------------------------+----------------------+
| 26 | Попов Влас Игнатьевич         |                      7 |           4128358.12 |
+----+-------------------------------+------------------------+----------------------+
|  2 | Кира Филипповна Калинина      |                      6 |           3885205.10 |
+----+-------------------------------+------------------------+----------------------+
| 41 | Вячеслав Арсенович Ефимов     |                      7 |           3682362.58 |
+----+-------------------------------+------------------------+----------------------+
| 37 | Субботин Пах

Получение статистики клиентов по возрастным группам

In [23]:
def get_clients_by_age_groups(session):
    age_groups = case(
        (Client.age < 25, "18-24"),
        (Client.age < 35, "25-34"),
        (Client.age < 45, "35-44"),
        (Client.age < 55, "45-54"),
        (Client.age < 65, "55-64"),
        (Client.age < 75, "65-74"),
        else_="75+"
    ).label('age_group')
    
    result = (
        session.query(
            age_groups,
            func.count(Client.idClient).label('clients_count'),
            func.count(Belay.idBelay).label('contracts_count'),
            func.avg(Belay.totalAmount).label('avg_amount')
        )
        .join(Belay, Belay.idClient == Client.idClient, isouter=True)
        .group_by(age_groups)
        .order_by(age_groups)
        .all()
    )
    return result

result = get_clients_by_age_groups(session)

table_data = []
for group, clients, contracts, avg_amount in result: 
    table_data.append([
        group,           # Возрастная группа
        clients,         # Количество клиентов
        contracts,       # Количество договоров
        avg_amount       # Средняя сумма по договороу
    ])

headers = ["Возрастная группа", "Клиентов", "Договоров", "Средняя сумма"]

print(tabulate(table_data[:10], headers=headers, tablefmt="grid", floatfmt=".2f"))

+---------------------+------------+-------------+-----------------+
| Возрастная группа   |   Клиентов |   Договоров |   Средняя сумма |
+=====================+============+=============+=================+
| 18-24               |         17 |          16 |       558345.18 |
+---------------------+------------+-------------+-----------------+
| 25-34               |         32 |          29 |       497328.53 |
+---------------------+------------+-------------+-----------------+
| 35-44               |         24 |          22 |       538641.02 |
+---------------------+------------+-------------+-----------------+
| 45-54               |         35 |          27 |       454375.63 |
+---------------------+------------+-------------+-----------------+
| 55-64               |         29 |          26 |       499553.40 |
+---------------------+------------+-------------+-----------------+
| 65-74               |         27 |          20 |       483154.22 |
+---------------------+-----------

Получение рейтинга филиалов по эффективности(сумма продаж на каждого агента)

In [24]:
def get_filials_efficiency(session):
    result = (
        session.query(
            Filial.title,
            func.count(Agent.idAgent).label('agents_count'),
            func.count(Belay.idBelay).label('contracts_count'),
            func.sum(Belay.totalAmount).label('total_amount'),
            (func.sum(Belay.totalAmount) / func.count(Agent.idAgent)).label('amount_per_agent')
        )
        .join(Agent, Agent.idFilial == Filial.idFilial, isouter=True)
        .join(Belay, Belay.idFilial == Filial.idFilial, isouter=True)
        .group_by(Filial.idFilial, Filial.title)
        .order_by(desc('amount_per_agent'))
        .all()
    )
    return result

result = get_filials_efficiency(session)

table_data = []
for title, agents, contracts, total, per_agent in result: 
    table_data.append([
        title,           # Возрастная группа
        agents,          # Количество агентов
        contracts,       # Количество договоров
        total,           # Общая сумма
        per_agent        # Сумма на агента
    ])

headers = ["Филиал", "Агентов", "Договоров", "Общая сумма", "Сумма на агента"]

print(tabulate(table_data[:10], headers=headers, tablefmt="grid", floatfmt=".2f"))

+-----------------------+-----------+-------------+---------------+-------------------+
| Филиал                |   Агентов |   Договоров |   Общая сумма |   Сумма на агента |
+=======================+===========+=============+===============+===================+
| ВСК (6)               |         3 |           3 |    2069149.79 |         689716.60 |
+-----------------------+-----------+-------------+---------------+-------------------+
| ВСК (8)               |         5 |           5 |    3146368.09 |         629273.62 |
+-----------------------+-----------+-------------+---------------+-------------------+
| ИНГОССТРАХ            |         5 |           5 |    3123418.71 |         624683.74 |
+-----------------------+-----------+-------------+---------------+-------------------+
| ВСК (4)               |         3 |           3 |    1845282.52 |         615094.17 |
+-----------------------+-----------+-------------+---------------+-------------------+
| ИНГОССТРАХ (5)        |       